In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import pickle
import os
import random
from shutil import copyfile

dataset_root = 'data'
train_split = 0.8

# Create directories for train and test data
train_data_dir = 'train'
test_data_dir = 'test'
os.makedirs(train_data_dir, exist_ok=True)
os.makedirs(test_data_dir, exist_ok=True)

In [2]:
for plant_category in os.listdir(dataset_root):
    if os.path.isdir(os.path.join(dataset_root, plant_category)):
        images = os.listdir(os.path.join(dataset_root, plant_category))
        num_images = len(images)
        num_train = int(train_split * num_images)

        random.shuffle(images)

        for i, image in enumerate(images):
            src = os.path.join(dataset_root, plant_category, image)
            if i < num_train:
                dst = os.path.join(train_data_dir, plant_category, image)
            else:
                dst = os.path.join(test_data_dir, plant_category, image)
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            copyfile(src, dst)


img_width, img_height = 224, 224  # ResNet50 input size
batch_size = 32

In [3]:
train_dir = 'train'
test_dir = 'test'

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1500 images belonging to 100 classes.
Found 700 images belonging to 100 classes.


In [5]:
# Load pre-trained ResNet50 model without top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Add custom top layers for your classification task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Save class indices
with open('class_indices.pkl', 'wb') as f:
    pickle.dump(train_generator.class_indices, f)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [19]:
# Train the model
epochs = 100
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator,
    callbacks=[early_stopping]
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)

# Save the model
model.save('plant_type_classifier_resnet.h5')

Epoch 1/100
38/38 [==============================] - 93s 2s/step - loss: 4.9875 - accuracy: 0.0158 - val_loss: 4.6213 - val_accuracy: 0.0050
Epoch 2/100
38/38 [==============================] - 107s 3s/step - loss: 4.7455 - accuracy: 0.0108 - val_loss: 4.5538 - val_accuracy: 0.0125
Epoch 3/100
38/38 [==============================] - 111s 3s/step - loss: 4.6745 - accuracy: 0.0167 - val_loss: 4.5132 - val_accuracy: 0.0325
Epoch 4/100
38/38 [==============================] - 118s 3s/step - loss: 4.6075 - accuracy: 0.0200 - val_loss: 4.4795 - val_accuracy: 0.0375
Epoch 5/100
38/38 [==============================] - 114s 3s/step - loss: 4.5715 - accuracy: 0.0175 - val_loss: 4.4443 - val_accuracy: 0.0500
Epoch 6/100
38/38 [==============================] - 117s 3s/step - loss: 4.5298 - accuracy: 0.0308 - val_loss: 4.4109 - val_accuracy: 0.0650
Epoch 7/100
38/38 [==============================] - 116s 3s/step - loss: 4.4962 - accuracy: 0.0308 - val_loss: 4.3818 - val_accuracy: 0.0525
Epoch 8

C:\Users\nicol\anaconda3\envs\AI\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
